I feel I have exhausted my efforts with random forests(which is not true), so here I will try various implentation of gradient Boosting trees from sklearn, XGBoost, LightGBM with differet feature selection measures, include feature interactions, deal with outliers and may be finally if it requires we can stack models. 

In [34]:
!pip install tqdm

    100% |████████████████████████████████| 61kB 4.5MB/s 


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from tqdm import tqdm
%matplotlib inline
rs = 100

In [0]:
train = pd.read_csv('drive/workspace/data/merc/train.csv')
test = pd.read_csv('drive/workspace/data/merc/test.csv')

In [0]:
cat_feat = list(train.select_dtypes(include=['object']).columns)
num_feat = list(train.select_dtypes(include=[np.number]).drop(['y'], axis=1).columns)

In [16]:
len(cat_feat), len(num_feat)

(8, 369)

So here cat_feat contains all categorical features, and num_feat has all numerical features excluding the target y. 

## Utility functions

Some of the utility functions are same as from nb2 but still here I am writing them again. 

In [0]:
def val_r2_score(estimator, X, y, n_splits=5, n_repeats=10, random_state=rs):
  val_scores = np.array([])
  trn_scores = np.array([])
  for i in tqdm(range(n_repeats)):
    kf = KFold(n_splits, True, random_state=rs+i)
    for ti, vi in kf.split(X):
      X_trn, X_val = X.iloc[ti], X.iloc[vi]
      y_trn, y_val = y.iloc[ti], y.iloc[vi]
      estimator.fit(X_trn, y_trn)
      val_scores = np.append(val_scores, r2_score(y_val, estimator.predict(X_val)))
      trn_scores = np.append(trn_scores, r2_score(y_trn, estimator.predict(X_trn)))
  return (val_scores.reshape((n_repeats, n_splits)), trn_scores.reshape( n_repeats, n_splits))

In [0]:
def me_val_r2_score(estimator, train, columns, n_splits=5, n_repeats=2, random_state=rs):
  val_scores = np.array([])
  for i in tqdm(range(n_repeats)) : 
    kf1 = KFold(n_splits=n_splits, shuffle=True, random_state=rs+i)
    for ti, vi in kf1.split(train):
      ttrain, vtrain = train.iloc[ti], train.iloc[vi]
      for col in columns: 
        means = ttrain.groupby(col)['y'].mean()
        ttrain[col+'_mt'] = ttrain[col].map(means)
        vtrain[col+'_mt'] = vtrain[col].map(means)
      
      vtrain.fillna(ttrain['y'].mean(), inplace=True)
      X_trn = ttrain
      X_val = vtrain
      
      kf2 = KFold(n_splits=5, shuffle=True, random_state=rs)
      for tind, vind in kf2.split(ttrain) : 
        ttrn, vtrn = ttrain.iloc[tind], ttrain.iloc[vind]
        for col in columns : 
          means = ttrn.groupby(col)['y'].mean()
          vtrn[col+'_mt'] = vtrn[col].map(means)
        X_trn.iloc[vind] = vtrn

      X_trn.fillna(ttrain['y'].mean(), inplace=True)
      
      _=estimator.fit(X_trn.drop(['y']+columns, axis=1), X_trn['y'])
      val_scores = np.append(val_scores, r2_score(X_val['y'], estimator.predict(X_val.drop(['y']+columns, axis=1))))
  return val_scores.reshape(n_repeats, n_splits)


In [0]:

def me_df(train, n_splits=4, n_repeats=10, random_state=rs):
  trns = []
  vals = []
  for i in tqdm(range(n_repeats)):
    kf1 = KFold(n_splits, True, random_state=rs+i)
    for ti, vi in kf1.split(train):
      ttrain, vtrain = train.iloc[ti], train.iloc[vi]
      for col in cat_feat:
        means = ttrain.groupby(col)['y'].mean()
        ttrain[col+'_mt'] = ttrain[col].map(means)
        vtrain[col+'_mt'] = vtrain[col].map(means)
      vtrain.fillna(ttrain['y'].mean(), inplace=True)
      
      X_trn = ttrain.copy()
      X_val = vtrain.copy()
      
      kf2 = KFold(5, True, random_state=rs)
      for tind, vind in kf2.split(ttrain):
        ttrn, vtrn = ttrain.iloc[tind], ttrain.iloc[vind]
        for col in cat_feat:
          means = ttrn.groupby(col)['y'].mean()
          vtrn[col+'_mt'] = vtrn[col].map(means)
        X_trn.iloc[vind] = vtrn
      X_trn.fillna(ttrain['y'].mean(), inplace=True)
      trns.append(X_trn)
      vals.append(X_val)
  return trns, vals


def me_r2_score(estimator, trns, vals, me_cols, drop_feat=[], random_state=rs):
  val_scores=[]
  ohe_cols = [col for col in cat_feat if col not in me_cols]
  for trn, val in tqdm(zip(trns, vals)) : 
    trn = trn.drop(me_cols, axis=1)
    val = val.drop(me_cols, axis=1)
    for col in ohe_cols : 
      trn = trn.drop([col+'_mt'], axis=1)
      val = val.drop([col+'_mt'], axis=1)
    if len(ohe_cols) : 
      trn = pd.get_dummies(trn, columns=ohe_cols)
      val = pd.get_dummies(val, columns=ohe_cols)
        
    if len(drop_feat) :
      trn = trn.drop([f for f in drop_feat if f in list(trn.columns)], axis=1)
      val = val.drop([f for f in drop_feat if f in list(val.columns)], axis=1)
    
    print (trn.shape, val.shape)
    
    _=estimator.fit(trn.drop(['y'],  axis=1), trn['y'])
    preds = estimator.predict(val.drop(['y'], axis=1))
    
    val_scores.append(r2_score(val['y'], preds))
  return val_scores
    


In [0]:
def validation_curve(estimator, trn, val, me_cols=cat_feat, drop_feat=[]):
    
    ohe_cols = [c for c in cat_feat if c not in me_cols]
    trn = trn.drop(me_cols, axis=1)
    val = val.drop(me_cols, axis=1)
    for col in ohe_cols : 
      trn = trn.drop([col+'_mt'], axis=1)
      val = val.drop([col+'_mt'], axis=1)
    if len(ohe_cols) : 
      trn = pd.get_dummies(trn, columns=ohe_cols)
      val = pd.get_dummies(val, columns=ohe_cols)
        
    if len(drop_feat) :
      trn = trn.drop([f for f in drop_feat if f in list(trn.columns)], axis=1)
      val = val.drop([f for f in drop_feat if f in list(val.columns)], axis=1)
    
    X_train = trn.drop(['y'], axis=1)
    X_val = val.drop(['y'], axis=1)
    y_train = trn['y']
    y_val = val['y']
    
    _ = estimator.fit(X_train, y_train)
    
    test_score = np.empty(len(estimator.estimators_))
    train_score = np.empty(len(estimator.estimators_))
    
    for i, pred in enumerate(estimator.staged_predict(X_val)):
        test_score[i] = r2_score(y_val, pred)

    for i, pred in enumerate(estimator.staged_predict(X_train)):
        train_score[i] = r2_score(y_train, pred)

    best_iter = np.argmax(test_score)
    
#     Uncommment the following lines to see a plot. 
    
    
#     test_line = plt.plot(test_score, label='test')

#     colour = test_line[-1].get_color()
#     plt.plot(train_score, '--', color=colour, label='train')

#     plt.xlabel("Number of boosting iterations")
#     plt.ylabel("R2 score")
#     plt.legend(loc='best')
#     plt.axvline(x=best_iter, color=colour)
    return best_iter

## Scikit Learn

Let's start with sklearn's implementation of GBM. 

In [0]:
X = train.drop(['y'], axis=1)
X = pd.get_dummies(X, columns=cat_feat)
y = train['y']

In [84]:
gbr = GradientBoostingRegressor()
val_scores, trn_scores = train_val_r2_score(gbr, X, y)

100%|██████████| 10/10 [01:20<00:00,  8.07s/it]


In [55]:
print(val_scores.mean(), val_scores.std())
print (trn_scores.mean(), trn_scores.std())
print (val_scores)

(0.5592712508115831, 0.05244097706310926)
(0.6577529734988582, 0.010593274705071789)
[[0.52770343 0.50505212 0.5545412  0.6141021  0.56593822]
 [0.60448107 0.61669633 0.51661907 0.5503775  0.47292347]
 [0.6047506  0.60202299 0.57138764 0.54706123 0.47375107]
 [0.5818624  0.43133665 0.58951768 0.56070441 0.60799904]
 [0.58047576 0.61070046 0.59302227 0.61685635 0.44035624]
 [0.55522127 0.58194286 0.62588408 0.5726197  0.47735789]
 [0.60163942 0.47447391 0.63212972 0.57288287 0.50519909]
 [0.52208136 0.57322103 0.5623261  0.51713715 0.60830957]
 [0.45906274 0.53498321 0.59982736 0.63724457 0.59071208]
 [0.58256642 0.5470017  0.61531224 0.49681789 0.577369  ]]


I will train a full vanilla GBM model and submit to see how it fares on leaderboard compared our score here. 

In [65]:
X_test = pd.get_dummies(test, columns=cat_feat)
X_test.shape, X.shape

((4209, 570), (4209, 564))

So there are some columns(ie some level of categorical columns) that are in test in not in train. So for the columns only in train we add them to test and for columns only in test we drop them. 

In [66]:
col1 = [f for f in X_test.columns if f not in X.columns]
col2 = [f for f in X.columns if f not in X_test.columns]
len(col1), len(col2)

(16, 10)

In [67]:
# We drop col1 from X_test and add col2 to X_test
X_test = X_test.drop(col1, axis=1)
new_df = pd.DataFrame(np.zeros((X_test.shape[0],len(col2))), index=X_test.index, columns=col2)
X_test = pd.concat([X_test, new_df], axis=1)
X_test.shape, X.shape

((4209, 564), (4209, 564))

In [70]:
gbr = GradientBoostingRegressor()
gbr.fit(X,y)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((X_test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm18.csv', subm, delimiter=',', header='ID,y', comments='', fmt=['%d','%.6f'])

Public LB : 0.52614, PrivateLB : 0.52965

Next instead of one hot encoding, we will target_encode our categories and then tune the GBM parameters. 

So first we get a list of trains and validation dataframes from multiple repeats of KFold in mean-encoded form, so that we don't have to to geenrate them everytime we test a new model with changed parameters. 


In [0]:
n_splits=4
n_repeats=10
trns, vals = me_df(train, n_splits, n_repeats)

In [150]:
gbr = GradientBoostingRegressor()
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat)).reshape((n_repeats, n_splits))

100%|██████████| 50/50 [02:04<00:00,  2.49s/it]


In [160]:
print (val_scores.mean(), val_scores.std())

(0.5648632147848137, 0.055460594449785315)


Let's tune our gbr model. 

First we will figure out the n_estimators for a sufficiently high learning rate. For this n_estimators and learning rate we tune other parameters and later we can lower learning rate and increase n_estimators. 

In [170]:
trns, vals = me_df(train)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [0]:
params = dict(n_estimators=500, max_depth=5, max_features='sqrt', learning_rate=0.1, subsample=.8, random_state=rs)
gbr = GradientBoostingRegressor(**params)

In [203]:
# validation_curve(gbr, trns[1], vals[1])
best_iters = []
for trn, val in tqdm(zip(trns, vals)): 
  best_iters.append(validation_curve(gbr, trn, val))

100%|██████████| 40/40 [01:41<00:00,  2.54s/it]


In [204]:
print (np.array(best_iters).mean(), np.array(best_iters).std())

(52.85, 13.047126120337767)


So we have n_estimators=53 where we have highest val_score for learning rate=0.1 . So we will stick with that and tune other params. Later we will also  delve into feature selection. But before we tune let's drop features that have near zero variance or which have only one predominant category like X4. 



In [0]:
low_var_feat = ['X4', 'X4_mt']
var_df = pd.DataFrame([train[col].sum() for col in num_feat], index=num_feat, columns=[ 'ones'])
var_df.sort_values(by='ones', inplace=True)

In [264]:
var_df[var_df['ones']<50].shape
low_var_feat.extend(list(var_df[var_df['ones']<50].index))
len(low_var_feat)

(153, 1)

155

So there are 153 such columns which have less than 50 ones out of 4209. So we added them to drop list. Next we check if there are some duplicates. 

In [0]:
d_trn= train.T.drop_duplicates().T
dup_feat = [f for f in train if f not in d_trn]

In [266]:
print (len(dup_feat))

56


So these 56 columns can also be dropped. We check our scores after dropping these features. 

In [262]:
n_splits=4
n_repeats=10
trns, vals = me_df(train, n_splits, n_repeats)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [0]:
gbr = GradientBoostingRegressor(random_state=rs)
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat, drop_feat=list(set(low_var_feat+dup_feat)))).reshape((n_repeats, n_splits))

In [279]:
print (val_scores.mean(), val_scores.std())

(0.5651195117270597, 0.049668760519681034)


Just miniscule improvement in score. Again let's find an optimal value for n_estimators. 

In [0]:
params = dict(n_estimators=300, random_state=rs)
gbr = GradientBoostingRegressor(**params)

In [0]:
best_iters = []
for trn, val in tqdm(zip(trns, vals)): 
  best_iters.append(validation_curve(gbr, trn, val, drop_feat=list(set(low_var_feat+dup_feat))))

In [285]:
print (np.array(best_iters).mean(), np.array(best_iters).std())

(49.45, 20.179135263930416)


Too high deviation. Let's take optimal value to be 70 so that we don't underfit and find scores. 

In [0]:
gbr = GradientBoostingRegressor(n_estimators=70, random_state=rs)
val_scores = np.array(me_r2_score(gbr, trns, vals, cat_feat, drop_feat=list(set(low_var_feat+dup_feat)))).reshape((n_repeats, n_splits))

In [288]:
print (val_scores.mean(), val_scores.std())

(0.5678109876250517, 0.049773153219372396)


No doubt there is some improvement, but this high variation is bugging me. Anyway let's submit this and see. 

In [0]:
me_cols = cat_feat
train_new = train.copy()
test_new = test.copy()
for col in me_cols : 
  means = train_new.groupby(col)['y'].mean()
  train_new[col+'_mt'] = train_new[col].map(means)
  test_new[col+'_mt'] = test_new[col].map(means)

test_new.fillna(train_new['y'].mean(), inplace=True)
 

In [299]:
# let's regularise train_new

train_new1 = train_new.copy()

kf = KFold(n_splits=5, shuffle=True, random_state=rs)
for ti, vi in kf.split(train_new):
  trn, val = train_new.iloc[ti], train_new.iloc[vi]
  for col in me_cols:
    means = trn.groupby(col)['y'].mean()
    val[col+'_mt'] = val[col].map(means)
  train_new1.iloc[vi] = val
train_new1.fillna(train_new['y'].mean(), inplace=True)
    

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
feat_to_drop = list(set(['y'] + cat_feat + dup_feat + low_var_feat))
X_trn = train_new1.drop(feat_to_drop, axis=1)
y_trn = train_new1['y']
X_test = test_new[X_trn.columns]

In [306]:
gbr = GradientBoostingRegressor(n_estimators=70, random_state=rs)
gbr.fit(X_trn, y_trn)
y_pred = gbr.predict(X_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=70, presort='auto', random_state=100,
             subsample=1.0, verbose=0, warm_start=False)

In [0]:
subm = np.stack((test.ID, y_pred), axis=1)
np.savetxt('drive/workspace/data/merc/subm19.csv', subm, delimiter=',', comments='', header='ID,y', fmt=['%d', '%.6f'])

Private LB : 0.54652  Public LB : 0.55235 . Okay decent score, 